# EDA for Youtube Comments
Calvin Dailey, Keenen Cates, Zeyu Zhang

In [18]:
import numpy as np
import pandas as pd
from requests import get
from os import path
from io import BytesIO
from zipfile import ZipFile
from emoji_list import all_emoji
from time import strftime
from urllib.request import urlopen 
from collections import Counter
from string import punctuation

excluded = ['1','2','3','4', 
            '5','6','7','8', 
            '9','0','#','*']
emojis = [e for e in all_emoji if e not in excluded]

today = strftime('%x').replace('/', '_')

data_url = 'https://www.kaggle.com/datasnaek/youtube/downloads/youtube.zip'
root_dir = '.'
data_root = path.join(root_dir, 'data')
data_path = path.join(data_root, 'data_' + today + '/')
test_data = path.join(data_root, 'data_10_03_17')

In [2]:
def download_and_extract_youtube_files():
    zipper = ZipFile(BytesIO(urlopen(data_url)))
    zipper.extractall(data_path)

In [10]:
def get_comments(path):
    df = pd.read_csv(path + '/UScomments.csv', error_bad_lines=False)
    comments = df['comment_text']
    
    words = Counter()
    for each in comments:
        words.update(''.join([c for c in str(each).lower() if c not in punctuation]).split(' '))
        
    print('The unique words are: ', len(words))
    print(words.most_common(10))
    return comments

comments = get_comments(test_data)

b'Skipping line 41589: expected 4 fields, saw 11\nSkipping line 51628: expected 4 fields, saw 7\nSkipping line 114465: expected 4 fields, saw 5\n'
b'Skipping line 142496: expected 4 fields, saw 8\nSkipping line 189732: expected 4 fields, saw 6\nSkipping line 245218: expected 4 fields, saw 7\n'
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


The unique words are:  155563
[('the', 200114), ('i', 134169), ('a', 116415), ('to', 115947), ('and', 107542), ('', 105545), ('you', 92741), ('is', 91158), ('this', 75285), ('of', 74897)]


In [14]:
def split_emoji_data(threshold=len(comments)):
    has_emoji = lambda s: any((True for x in [c for c in str(s)] if x in emojis))
    
    emoji_comments = []
    no_emoji_comments = []
    for each in comments[:threshold]:
        if has_emoji(each):
            emoji_comments.append(each)
        else:
            no_emoji_comments.append(each)
            
    return emoji_comments, no_emoji_comments

emoji_comments, no_emoji_comments = split_emoji_data()

In [15]:
emoji_comments

["Logan Paul it's yo big day ‼️‼️‼️",
 'trending 😉',
 'The end though 😭👍🏻❤️',
 'I love Logan and Jake so much and thay are so amazing and I look up to them so much 😍💋💙👍',
 "👈 watch by clicking here you can see people's are entertaining",
 'Evan is being a douch Logans getting pissed, Logan should have went to smash his game system then evan would have cried 😂 & Evans not gonna be able to take care of a pitbull tell his ass no, its gonna kill kong !!!',
 "When I saw this vid at first I was like naaahhhhh to tired but I looked at it again and saw the title and I was like H🔥LLLLLLZZZZ YEEEAAAHHHH and as a anniversary present Logan Paul I'm gonna b u for Halloween to show how much u hav changed ma life and I'm not even joking about it 😂😂😂",
 'The girl with the dog was kylie Jenner 😂',
 'Attack of titan squad were u at👻❤️😂',
 'Congrats Logan love u so much someday I would love to meet u #1 fan of the logang ❤️',
 'Me and my friends subscribe AAAAAAAAAAAA you got the Diamond Play button 💎😆😆😆

In [31]:
def extract_emoji_target():
    inputs = [] 
    targets = []
    for comment in emoji_comments:
        x = []
        y = []
        for c in comment:
            if c in emojis:
                y.append(c)
            else:
                x.append(c)
        inputs.append(''.join(x))
        targets.append(''.join(set(y)))
    return inputs, targets

inputs, labels = extract_emoji_target()

In [35]:
list(zip(inputs, labels))

[("Logan Paul it's yo big day ", '️‼'),
 ('trending ', '😉'),
 ('The end though ', '🏻😭❤️👍'),
 ('I love Logan and Jake so much and thay are so amazing and I look up to them so much ',
  '💙💋😍👍'),
 (" watch by clicking here you can see people's are entertaining", '👈'),
 ('Evan is being a douch Logans getting pissed, Logan should have went to smash his game system then evan would have cried  & Evans not gonna be able to take care of a pitbull tell his ass no, its gonna kill kong !!!',
  '😂'),
 ("When I saw this vid at first I was like naaahhhhh to tired but I looked at it again and saw the title and I was like HLLLLLLZZZZ YEEEAAAHHHH and as a anniversary present Logan Paul I'm gonna b u for Halloween to show how much u hav changed ma life and I'm not even joking about it ",
  '😂🔥'),
 ('The girl with the dog was kylie Jenner ', '😂'),
 ('Attack of titan squad were u at', '😂❤️👻'),
 ('Congrats Logan love u so much someday I would love to meet u #1 fan of the logang ',
  '❤️'),
 ('Me and my frie